So this is my weather and travel assistant. 
I specify date and more info like number of days to it - and it will suggest me where I can go. I can tell it budget. And it will get flight prices. And I can ask if its good to go to this particular place on this date - and it will recommend an answer based on weather conditions there for that day. 


#### Weather + Travel planner 
     Combine tools:
     get_weather(city)
     get_flight_price(city)
     recommend_travel(city)


So I have to select a few cities my airline goes to. Then dates and times of each flight. - some flights may not go to some cities on a particular dates or whatever. - Make a db of this. 

In [62]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [63]:
from datetime import datetime

In [64]:
import psycopg

In [65]:
load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI()
weather_api_key = os.getenv('WEATHER_API_KEY')
print(weather_api_key)

66da7414e5ee28dbae1a674f6e581248


In [66]:
system_prompt = "You are a helpful assistant who talks politely to the user."

In [5]:
def chat(message, history):
    history =  [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role":"system", "content":system_prompt}] + history + [{"role":"user", "content":message}]
    responses = openai.chat.completions.create(model = "gpt-4.1-mini", messages=messages)
    return responses.choices[0].message.content

In [6]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [67]:
import requests


In [19]:
data.keys()

dict_keys(['cod', 'message', 'cnt', 'list', 'city'])

In [35]:
# [k for k in data['list'] if '2026-02-26'  in k['dt_txt'] ]

# Lets get started with function and the tool creation. 

Define exactly what this assistant can do:

Provide current weather

Provide 5-day forecast

Suggest flights by:

destination city

optional date

Combine weather + flights

Reject dates > 6 days

Handle missing date (assume today)

If you don’t define this clearly, the LLM will behave inconsistently.

In [139]:
current_date =  datetime.now().strftime('%Y-%m-%d')
system_prompt = f"""
You are a helpful, polite, and intelligent Weather + Travel Assistant.

Your Responsibilities:

1. Weather Forecast:
- When the user asks about weather for a specific city and date:
    - Call the get_weather function.
    - When user asks for weather, then only for 'from_city' - only if it is safe to fly or not should be told. 
    - For destination city, all weather details must be summarized. 
    - If no date is provided, assume today's date: {current_date}.
    - If the date is more than 4 days after {current_date}, politely inform the user that weather forecast is only available for the next 5 days.
    - Summarize the weather in a clear and friendly way and Suggest what kind of clothes to pack (warm clothes, umbrella, summer wear, etc.).
    - If weather indicates extreme or high alert conditions, advise the user not to travel.

2. Travel Planning:
- If the user asks about traveling between cities:
    - Provide weather advice.
    - Ask if they would like flight details.
    - If requested, call the get_flights function.

3. Flight Information:
- If flight data is not found:
    - Politely inform the user that flight details are available only for limited dates (next 4 days) and only for certain cities.
    - Do not make up flight data.

General Rules:
- Be polite and conversational.
- Never hallucinate weather or flight data.
- Always use available functions when required.
- Do not answer weather questions without calling the function.

Decision Logic:

1. If user asks about travel safety:
    → Always check weather first.
    → Only if weather is safe, then offer or provide flight details.
    → If weather cannot be determined, do NOT provide flights.

2. If weather data is unavailable for requested date:
    → Ask the user to clarify a specific date within next 5 days.
    → Do not assume or hallucinate.

3. When user says "this weekend":
    → Interpret it as the upcoming Saturday and Sunday relative to current date.
    → If today is Thursday (2026-02-26), weekend = 2026-02-28 and 2026-03-01.
"""


In [140]:
get_weather_function = {
    "name": "get_weather",
    "description": "Get weather forecast for a specific city. If no date is specified then the function returns current date's forecast",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "City name"
            },
            "date": {
                "type": "string",
                "description": "Date in YYYY-MM-DD format"
            }
        },
        "required": ["city"]
    }
}
get_flight_function =  {
    "name": "get_flights",
    "description": "Get flight details for a given departure city and destination city. If date is provided, return flights for that specific date. If date is not provided, return all available upcoming flights between the cities.",
    "parameters": {
        "type": "object",
        "properties": {
            "from_city": {
                "type": "string",
                "description": "Departure City name"
            },
            "to_city": {
                "type": "string",
                "description": "Destination City name"
            },
            "date": {
                "type": "string",
                "description": "Date in YYYY-MM-DD format (option)"
            }
        },
        "required": ["from_city", "to_city"]
    }
}

In [141]:
def get_weather(city, date=None):
    print(f"In get_weather - with {city}, and {date}")
    if date:
        pass
    else: 
         date = datetime.now().strftime('%Y-%m-%d')
    url = f"https://api.openweathermap.org/data/2.5/forecast?q={city}&appid={weather_api_key}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        filtered = [
        {
            "time": entry["dt_txt"],
            "temperature": entry["main"]["temp"],
            "description": entry["weather"][0]["description"],
            "rain_probability": entry.get("pop", 0)
        }
        for entry in data["list"]
        if date in entry["dt_txt"]
        ]
        data = {"city":city, "date":date, "forecast":filtered}
    else:
        return "Error"
    return data

In [142]:
now = datetime.now().strftime('%Y-%m-%d')
get_weather('Delhi',now)

In get_weather - with Delhi, and 2026-02-26


{'city': 'Delhi',
 'date': '2026-02-26',
 'forecast': [{'time': '2026-02-26 18:00:00',
   'temperature': 22.42,
   'description': 'clear sky',
   'rain_probability': 0},
  {'time': '2026-02-26 21:00:00',
   'temperature': 21.94,
   'description': 'clear sky',
   'rain_probability': 0}]}

In [143]:
def get_flights(from_city, to_city, date=None):
    if date:
        with psycopg.connect("dbname=MyRestaurantDB user=postgres password=Veenamaa@2103") as conn:
            with conn.cursor() as cur:
                cur.execute("SELECT * FROM flights_detail where date=%s and from_city=%s and to_city=%s;", (date, from_city, to_city))
                rows = cur.fetchall()
    else:
        with psycopg.connect("dbname=MyRestaurantDB user=postgres password=Veenamaa@2103") as conn:
            with conn.cursor() as cur:
                cur.execute("SELECT * FROM flights_detail where from_city=%s and to_city=%s;", (from_city, to_city))
                rows = cur.fetchall()
    return rows

In [144]:
tools = [{"type": "function", "function": get_weather_function},{"type": "function", "function": get_flight_function}]

In [145]:
available_functions = { "get_weather": get_weather, "get_flights": get_flights }
def chat(message, history):
    print("in chat")
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages, tools=tools)
    
    if response.choices[0].finish_reason == "tool_calls":
        print("In if statement of chat")
        message = response.choices[0].message
        response = handle_tool_calls(message)
        messages.append(message)
        messages.extend(response)
        response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
    
    return response.choices[0].message.content

In [146]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        function_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        function_to_call = available_functions.get(function_name)
        if function_to_call:
            print("In if of handle")
            result = function_to_call(**arguments)
            responses.append({
                "role": "tool",
                "content": str(result),
                "tool_call_id": tool_call.id
            })
            print(responses)
    return responses

In [147]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


in chat
In if statement of chat
In if of handle
In get_weather - with Delhi, and 2026-02-28
[{'role': 'tool', 'content': "{'city': 'Delhi', 'date': '2026-02-28', 'forecast': [{'time': '2026-02-28 00:00:00', 'temperature': 21.65, 'description': 'scattered clouds', 'rain_probability': 0}, {'time': '2026-02-28 03:00:00', 'temperature': 23.48, 'description': 'scattered clouds', 'rain_probability': 0}, {'time': '2026-02-28 06:00:00', 'temperature': 30.58, 'description': 'few clouds', 'rain_probability': 0}, {'time': '2026-02-28 09:00:00', 'temperature': 33.39, 'description': 'clear sky', 'rain_probability': 0}, {'time': '2026-02-28 12:00:00', 'temperature': 31.86, 'description': 'clear sky', 'rain_probability': 0}, {'time': '2026-02-28 15:00:00', 'temperature': 27.68, 'description': 'clear sky', 'rain_probability': 0}, {'time': '2026-02-28 18:00:00', 'temperature': 24.31, 'description': 'clear sky', 'rain_probability': 0}, {'time': '2026-02-28 21:00:00', 'temperature': 22.33, 'description':

In [127]:
get_weather("Paris")

{'city': 'Paris',
 'date': '2026-02-26',
 'forecast': [{'time': '2026-02-26 18:00:00',
   'temperature': 15.65,
   'description': 'overcast clouds',
   'rain_probability': 0},
  {'time': '2026-02-26 21:00:00',
   'temperature': 15.08,
   'description': 'broken clouds',
   'rain_probability': 0}]}